# Tuberculosis Diagnosis using Transfer Learning
This notebook is written and executed by **Dr Raheel Siddiqi** on *05-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 5-fold cross validation is used to evaluate the model.

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.13.1
Keras Version:  2.2.4-tf


In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model   

In [3]:
image_height = 100
image_width = 100
batch_size = 2
no_of_epochs  = 15

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
%matplotlib inline

DATADIR = "D:\\TensorFlow Programs\\TB Diagnosis\\ChinaSet_AllFiles\\ChinaSet_AllFiles\\ChinaSetImages"

CATEGORIES = ["NORMAL", "TB"]

data=[]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (image_height, image_width))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
            img_array = img_array.astype(np.float32)/255.
            data.append([img_array, class_num])
        except Exception as e:   
            pass

print(len(data))

100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:34<00:00,  7.56it/s]


662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

0
1
1
0
0
1
1
0
1
1


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


In [7]:
print(len(X))

662


In [8]:
print(X.shape)

(662, 100, 100, 3)


In [9]:
k=5
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=4,epochs=10) # 10 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])
print('Average Validation Score: ', np.average(validation_scores))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
530/530 [==============================] - ETA: 7:45 - loss: 0.8208 - acc: 0.250 - ETA: 3:57 - loss: 0.9731 - acc: 0.375 - ETA: 2:41 - loss: 0.8693 - acc: 0.583 - ETA: 2:04 - loss: 0.8296 - acc: 0.562 - ETA: 1:41 - loss: 0.7807 - acc: 0.600 - ETA: 1:26 - loss: 0.7792 - acc: 0.583 - ETA: 1:15 - loss: 0.8073 - acc: 0.500 - ETA: 1:07 - loss: 0.7977 - acc: 0.468 - ETA: 1:00 - loss: 0.7848 - acc: 0.527 - ETA: 55s - loss: 0.7762 - acc: 0.525 - ETA: 51s - loss: 0.7606 - acc: 0.54 - ETA: 47s - loss: 0.7577 - acc: 0.54 - ETA: 44s - loss: 0.7575 - acc: 0.53 - ETA: 41s - loss: 0.7534 - acc: 0.53 - ETA: 39s - loss: 0.7502 - acc: 0.51 - ETA: 37s - loss: 0.7479 - acc: 0.50 - ETA: 35s - loss: 0.7442 - acc: 0.50 - ETA: 33s - loss: 0.7393 - acc: 0.51 - ETA: 32s - loss: 0.7321 - acc: 0.52 - ETA: 30s - loss: 0.7451 - acc: 0.51 - ETA: 29s - loss: 0.7288 - acc: 0.53 - ETA: 28s 

530/530 [==============================] - ETA: 11s - loss: 0.7031 - acc: 0.25 - ETA: 12s - loss: 0.5907 - acc: 0.50 - ETA: 12s - loss: 0.4904 - acc: 0.66 - ETA: 12s - loss: 0.4297 - acc: 0.75 - ETA: 11s - loss: 0.4508 - acc: 0.70 - ETA: 11s - loss: 0.4234 - acc: 0.75 - ETA: 11s - loss: 0.4547 - acc: 0.75 - ETA: 11s - loss: 0.5427 - acc: 0.68 - ETA: 11s - loss: 0.5575 - acc: 0.66 - ETA: 11s - loss: 0.5599 - acc: 0.67 - ETA: 11s - loss: 0.5733 - acc: 0.63 - ETA: 11s - loss: 0.5857 - acc: 0.64 - ETA: 11s - loss: 0.5702 - acc: 0.67 - ETA: 11s - loss: 0.6088 - acc: 0.64 - ETA: 11s - loss: 0.6161 - acc: 0.63 - ETA: 11s - loss: 0.6130 - acc: 0.64 - ETA: 11s - loss: 0.6146 - acc: 0.64 - ETA: 10s - loss: 0.6131 - acc: 0.65 - ETA: 10s - loss: 0.6134 - acc: 0.65 - ETA: 10s - loss: 0.6084 - acc: 0.66 - ETA: 10s - loss: 0.5931 - acc: 0.67 - ETA: 10s - loss: 0.5905 - acc: 0.68 - ETA: 10s - loss: 0.6216 - acc: 0.66 - ETA: 10s - loss: 0.6246 - acc: 0.65 - ETA: 10s - loss: 0.6258 - acc: 0.66 - ETA: 10

530/530 [==============================] - ETA: 11s - loss: 0.4535 - acc: 0.75 - ETA: 12s - loss: 0.7333 - acc: 0.50 - ETA: 12s - loss: 0.5958 - acc: 0.66 - ETA: 12s - loss: 0.6475 - acc: 0.62 - ETA: 12s - loss: 0.5993 - acc: 0.65 - ETA: 12s - loss: 0.6489 - acc: 0.54 - ETA: 12s - loss: 0.6894 - acc: 0.50 - ETA: 11s - loss: 0.6647 - acc: 0.56 - ETA: 11s - loss: 0.6451 - acc: 0.58 - ETA: 11s - loss: 0.6672 - acc: 0.57 - ETA: 11s - loss: 0.6505 - acc: 0.59 - ETA: 11s - loss: 0.6486 - acc: 0.58 - ETA: 11s - loss: 0.6270 - acc: 0.61 - ETA: 11s - loss: 0.6351 - acc: 0.60 - ETA: 11s - loss: 0.6181 - acc: 0.63 - ETA: 11s - loss: 0.6164 - acc: 0.64 - ETA: 11s - loss: 0.6068 - acc: 0.64 - ETA: 10s - loss: 0.6202 - acc: 0.62 - ETA: 10s - loss: 0.6070 - acc: 0.64 - ETA: 10s - loss: 0.6101 - acc: 0.63 - ETA: 10s - loss: 0.6037 - acc: 0.64 - ETA: 10s - loss: 0.5934 - acc: 0.65 - ETA: 10s - loss: 0.5904 - acc: 0.66 - ETA: 10s - loss: 0.5872 - acc: 0.66 - ETA: 10s - loss: 0.5837 - acc: 0.67 - ETA: 10

530/530 [==============================] - ETA: 11s - loss: 0.2748 - acc: 1.00 - ETA: 12s - loss: 0.4566 - acc: 0.87 - ETA: 11s - loss: 0.3694 - acc: 0.91 - ETA: 11s - loss: 0.4249 - acc: 0.87 - ETA: 12s - loss: 0.4002 - acc: 0.85 - ETA: 12s - loss: 0.3749 - acc: 0.87 - ETA: 11s - loss: 0.4195 - acc: 0.85 - ETA: 11s - loss: 0.4063 - acc: 0.84 - ETA: 11s - loss: 0.3883 - acc: 0.86 - ETA: 11s - loss: 0.4039 - acc: 0.85 - ETA: 11s - loss: 0.4387 - acc: 0.84 - ETA: 11s - loss: 0.4122 - acc: 0.85 - ETA: 11s - loss: 0.3901 - acc: 0.86 - ETA: 11s - loss: 0.4033 - acc: 0.85 - ETA: 11s - loss: 0.4188 - acc: 0.85 - ETA: 11s - loss: 0.4031 - acc: 0.85 - ETA: 11s - loss: 0.3896 - acc: 0.86 - ETA: 10s - loss: 0.3756 - acc: 0.87 - ETA: 10s - loss: 0.3752 - acc: 0.86 - ETA: 10s - loss: 0.3790 - acc: 0.85 - ETA: 10s - loss: 0.3705 - acc: 0.85 - ETA: 10s - loss: 0.3763 - acc: 0.85 - ETA: 10s - loss: 0.3747 - acc: 0.84 - ETA: 10s - loss: 0.3895 - acc: 0.84 - ETA: 10s - loss: 0.4223 - acc: 0.83 - ETA: 10

530/530 [==============================] - ETA: 11s - loss: 0.3443 - acc: 1.00 - ETA: 12s - loss: 0.4616 - acc: 0.75 - ETA: 11s - loss: 0.3740 - acc: 0.83 - ETA: 11s - loss: 0.3108 - acc: 0.87 - ETA: 11s - loss: 0.3708 - acc: 0.85 - ETA: 11s - loss: 0.3939 - acc: 0.83 - ETA: 11s - loss: 0.3538 - acc: 0.85 - ETA: 11s - loss: 0.3675 - acc: 0.84 - ETA: 11s - loss: 0.3600 - acc: 0.83 - ETA: 11s - loss: 0.3959 - acc: 0.80 - ETA: 11s - loss: 0.3711 - acc: 0.81 - ETA: 11s - loss: 0.3939 - acc: 0.81 - ETA: 11s - loss: 0.3751 - acc: 0.82 - ETA: 11s - loss: 0.3779 - acc: 0.82 - ETA: 11s - loss: 0.3560 - acc: 0.83 - ETA: 10s - loss: 0.3447 - acc: 0.84 - ETA: 10s - loss: 0.3498 - acc: 0.83 - ETA: 10s - loss: 0.3377 - acc: 0.84 - ETA: 10s - loss: 0.3278 - acc: 0.85 - ETA: 10s - loss: 0.3193 - acc: 0.86 - ETA: 10s - loss: 0.3394 - acc: 0.85 - ETA: 10s - loss: 0.3261 - acc: 0.86 - ETA: 10s - loss: 0.3210 - acc: 0.86 - ETA: 10s - loss: 0.3336 - acc: 0.86 - ETA: 10s - loss: 0.3700 - acc: 0.84 - ETA: 10

...

530/530 [==============================] - ETA: 15s - loss: 0.2097 - acc: 1.00 - ETA: 15s - loss: 0.2776 - acc: 1.00 - ETA: 15s - loss: 0.2371 - acc: 1.00 - ETA: 15s - loss: 0.2388 - acc: 1.00 - ETA: 15s - loss: 0.2928 - acc: 0.95 - ETA: 15s - loss: 0.3737 - acc: 0.87 - ETA: 15s - loss: 0.3497 - acc: 0.89 - ETA: 15s - loss: 0.3356 - acc: 0.90 - ETA: 14s - loss: 0.3120 - acc: 0.91 - ETA: 14s - loss: 0.3538 - acc: 0.87 - ETA: 14s - loss: 0.3663 - acc: 0.86 - ETA: 14s - loss: 0.3501 - acc: 0.87 - ETA: 14s - loss: 0.3446 - acc: 0.88 - ETA: 14s - loss: 0.3296 - acc: 0.89 - ETA: 14s - loss: 0.3220 - acc: 0.90 - ETA: 14s - loss: 0.4143 - acc: 0.87 - ETA: 14s - loss: 0.3990 - acc: 0.88 - ETA: 13s - loss: 0.4126 - acc: 0.87 - ETA: 13s - loss: 0.4558 - acc: 0.85 - ETA: 13s - loss: 0.4425 - acc: 0.86 - ETA: 13s - loss: 0.4261 - acc: 0.86 - ETA: 13s - loss: 0.4237 - acc: 0.86 - ETA: 13s - loss: 0.4319 - acc: 0.85 - ETA: 13s - loss: 0.4203 - acc: 0.86 - ETA: 13s - loss: 0.4275 - acc: 0.86 - ETA: 12

530/530 [==============================] - ETA: 15s - loss: 0.3125 - acc: 0.75 - ETA: 15s - loss: 0.2900 - acc: 0.87 - ETA: 15s - loss: 0.2622 - acc: 0.91 - ETA: 15s - loss: 0.2349 - acc: 0.93 - ETA: 15s - loss: 0.3182 - acc: 0.90 - ETA: 15s - loss: 0.3209 - acc: 0.91 - ETA: 15s - loss: 0.2922 - acc: 0.92 - ETA: 15s - loss: 0.3258 - acc: 0.90 - ETA: 15s - loss: 0.3407 - acc: 0.88 - ETA: 14s - loss: 0.3256 - acc: 0.90 - ETA: 14s - loss: 0.3553 - acc: 0.88 - ETA: 14s - loss: 0.3726 - acc: 0.87 - ETA: 14s - loss: 0.3567 - acc: 0.88 - ETA: 14s - loss: 0.4026 - acc: 0.87 - ETA: 14s - loss: 0.4104 - acc: 0.86 - ETA: 14s - loss: 0.4411 - acc: 0.84 - ETA: 14s - loss: 0.4677 - acc: 0.82 - ETA: 13s - loss: 0.4729 - acc: 0.81 - ETA: 13s - loss: 0.4812 - acc: 0.80 - ETA: 13s - loss: 0.4786 - acc: 0.80 - ETA: 13s - loss: 0.4644 - acc: 0.80 - ETA: 13s - loss: 0.4642 - acc: 0.80 - ETA: 13s - loss: 0.4651 - acc: 0.80 - ETA: 13s - loss: 0.4534 - acc: 0.81 - ETA: 13s - loss: 0.4560 - acc: 0.81 - ETA: 12

530/530 [==============================] - ETA: 15s - loss: 0.3606 - acc: 0.75 - ETA: 15s - loss: 0.3597 - acc: 0.75 - ETA: 15s - loss: 0.3809 - acc: 0.75 - ETA: 15s - loss: 0.3263 - acc: 0.81 - ETA: 15s - loss: 0.3374 - acc: 0.80 - ETA: 15s - loss: 0.3339 - acc: 0.79 - ETA: 15s - loss: 0.4030 - acc: 0.75 - ETA: 15s - loss: 0.3789 - acc: 0.78 - ETA: 14s - loss: 0.3749 - acc: 0.77 - ETA: 14s - loss: 0.3685 - acc: 0.77 - ETA: 14s - loss: 0.3608 - acc: 0.79 - ETA: 14s - loss: 0.4005 - acc: 0.75 - ETA: 14s - loss: 0.3754 - acc: 0.76 - ETA: 14s - loss: 0.3803 - acc: 0.76 - ETA: 14s - loss: 0.3697 - acc: 0.78 - ETA: 14s - loss: 0.3534 - acc: 0.79 - ETA: 13s - loss: 0.3359 - acc: 0.80 - ETA: 13s - loss: 0.3571 - acc: 0.79 - ETA: 13s - loss: 0.3410 - acc: 0.80 - ETA: 13s - loss: 0.3339 - acc: 0.81 - ETA: 13s - loss: 0.3333 - acc: 0.80 - ETA: 13s - loss: 0.4494 - acc: 0.78 - ETA: 13s - loss: 0.4310 - acc: 0.79 - ETA: 13s - loss: 0.4311 - acc: 0.79 - ETA: 13s - loss: 0.4281 - acc: 0.79 - ETA: 12

530/530 [==============================] - ETA: 15s - loss: 0.3812 - acc: 0.75 - ETA: 15s - loss: 0.4148 - acc: 0.75 - ETA: 15s - loss: 0.5485 - acc: 0.66 - ETA: 15s - loss: 0.4624 - acc: 0.75 - ETA: 15s - loss: 0.4509 - acc: 0.75 - ETA: 15s - loss: 0.4608 - acc: 0.75 - ETA: 15s - loss: 0.4467 - acc: 0.75 - ETA: 15s - loss: 0.4082 - acc: 0.78 - ETA: 15s - loss: 0.3922 - acc: 0.80 - ETA: 14s - loss: 0.4123 - acc: 0.77 - ETA: 14s - loss: 0.4457 - acc: 0.75 - ETA: 14s - loss: 0.4501 - acc: 0.75 - ETA: 14s - loss: 0.4638 - acc: 0.73 - ETA: 14s - loss: 0.4564 - acc: 0.73 - ETA: 14s - loss: 0.4379 - acc: 0.75 - ETA: 14s - loss: 0.4358 - acc: 0.76 - ETA: 14s - loss: 0.4446 - acc: 0.75 - ETA: 13s - loss: 0.4337 - acc: 0.76 - ETA: 13s - loss: 0.4206 - acc: 0.77 - ETA: 13s - loss: 0.4241 - acc: 0.77 - ETA: 13s - loss: 0.4228 - acc: 0.78 - ETA: 13s - loss: 0.4257 - acc: 0.78 - ETA: 13s - loss: 0.4284 - acc: 0.78 - ETA: 13s - loss: 0.4370 - acc: 0.78 - ETA: 13s - loss: 0.4476 - acc: 0.78 - ETA: 12

132/132 [==============================] - ETA: 0s - loss: 0.4059 - acc: 0.750 - ETA: 0s - loss: 0.4114 - acc: 0.828 - ETA: 0s - loss: 0.3979 - acc: 0.833 - ETA: 0s - loss: 0.3554 - acc: 0.867 - 1s 5ms/sample - loss: 0.3636 - acc: 0.8561
Average Validation Score:  0.8333333


In [10]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.8333333
